# AFAP Solvency Engine Validation

## Purpose of This Notebook

This notebook validates the **AFAP Phase 3 Solvency Engine**, which evaluates a company’s
capital structure and debt-servicing capacity using deterministic ratio logic.

The objectives of this notebook are to:

1. Verify that required solvency ratios are produced by the Ratio Engine Core
2. Confirm correct flag triggering and severity classification
3. Ensure schema compliance and orchestrator readiness
4. Provide an auditable, human-readable validation artifact

This notebook **does not modify engine logic**.  
It exists solely to validate correctness and integration readiness.


## AFAP Architectural Context

Within AFAP Phase 3:

- `ratio_engine_core` computes standardized financial ratios
- `solvency_engine` evaluates leverage and coverage risk
- `schema_validator` enforces output contracts
- The **orchestrator** aggregates engine outputs
- The **LLM layer** interprets already-validated results

This notebook simulates how the orchestrator will invoke the Solvency Engine
in a production pipeline.


In [2]:
# Environment setup and imports

import pandas as pd
import sys
import os

# Allow notebook to import AFAP engines
sys.path.append(os.path.abspath('..'))

from engines.ratio_engine_core import ratio_engine
from engines.solvency_engine import solvency_engine


## Load Clean Financial Statement Data

AFAP engines operate exclusively on cleaned, normalized financial statements.

This dataset has already passed:
- Structural normalization
- Line-item harmonization
- Missing value handling


In [3]:
financials = pd.read_csv("../data/cleaned/financial_statements.csv")
financials.head()


,Company,Year,FS Category,FS Subcategory,Statement,Amount
0,Acme Manufacturing Ltd,2020,Assets,Current Assets,Balance Sheet,3109667
1,Acme Manufacturing Ltd,2020,Assets,Non-Current Assets,Balance Sheet,905812
2,Acme Manufacturing Ltd,2020,Equity,Equity,Balance Sheet,2152630
3,Acme Manufacturing Ltd,2020,Expenses,COGS,Income Statement,373114
4,Acme Manufacturing Ltd,2020,Expenses,Finance Costs,Income Statement,2929304


## Run Ratio Engine Core

The Solvency Engine does **not** calculate ratios itself.
It relies on the Ratio Engine Core as a single source of truth.

This separation ensures:
- Deterministic behavior
- Cross-engine consistency
- Auditability


In [4]:
ratios_core = ratio_engine(financials)
ratios_df = pd.DataFrame(ratios_core)
ratios_df.head()


✅ ratio_engine output validated successfully.


,engine,Company,Year,metrics,flags,severity,explanation
0,ratio_engine,Acme Manufacturing Ltd,2020,"{'current_ratio': 1.5876421245028514, 'quick_r...",{},stable,Canonical financial ratios
1,ratio_engine,Acme Manufacturing Ltd,2021,"{'current_ratio': 3.0794864428132476, 'quick_r...",{},stable,Canonical financial ratios
2,ratio_engine,Acme Manufacturing Ltd,2022,"{'current_ratio': 3.4109333800409956, 'quick_r...",{},stable,Canonical financial ratios
3,ratio_engine,Acme Manufacturing Ltd,2023,"{'current_ratio': 4.67288599769455, 'quick_rat...",{},stable,Canonical financial ratios
4,ratio_engine,Banyan Retail Co,2020,"{'current_ratio': 2.513537113097586, 'quick_ra...",{},stable,Canonical financial ratios


## Flatten Nested Metrics for Validation

Many AFAP engines store numeric ratios under a `"metrics"` dictionary.  
To validate or use them in the Solvency Engine, we must **flatten this structure**.


In [6]:
# Flatten 'metrics' column into separate columns
metrics_df = pd.json_normalize(ratios_df['metrics'])
ratios_flat = pd.concat([ratios_df[['Company', 'Year']], metrics_df], axis=1)

ratios_flat.head()


,Company,Year,current_ratio,quick_ratio,gross_margin,operating_margin,net_margin,debt_equity,interest_coverage,asset_turnover,roa,roe
0,Acme Manufacturing Ltd,2020,1.587642,1.587642,0.867371,0.289107,-1.546148,0.982524,0.277650,0.700595,-1.083224,-2.020627
1,Acme Manufacturing Ltd,2021,3.079486,3.079486,0.854884,0.212159,-0.787051,1.358428,0.388825,0.772164,-0.607732,-2.132176
2,Acme Manufacturing Ltd,2022,3.410933,3.410933,0.968294,0.680335,0.301875,0.696958,2.446630,1.990334,0.600832,1.548439
3,Acme Manufacturing Ltd,2023,4.672886,4.672886,0.771416,0.510596,0.182190,1.476542,2.003211,1.965558,0.358106,1.137251
4,Banyan Retail Co,2020,2.513537,2.513537,0.757082,0.632036,0.180853,1.047567,2.023443,3.234804,0.585025,0.796404


## Validate Required Solvency Metrics

The Solvency Engine requires:

- `debt_equity`  
- `interest_coverage`

Optional metrics for sanity checking:

- `roa`  
- `roe`


In [7]:
for col in ["debt_equity", "interest_coverage", "roa", "roe"]:
    print(col, "min:", ratios_flat[col].min(), "max:", ratios_flat[col].max())


debt_equity min: 0.6654615509665709 max: 9.052038324909445
interest_coverage min: -1.3247712442403285 max: 2.9276479739936097
roa min: -2.428203525780877 max: 0.6008322161557166
roe min: -6.333509146352113 max: 1.5484389021911336


## Run Solvency Engine

The Solvency Engine evaluates **capital structure risk** using locked thresholds:

- High leverage → `debt_equity > 1.5`  
- Weak coverage → `interest_coverage < 1.5`  

Severity rules:

- `stable` → no flags  
- `watch` → one flag  
- `action` → both flags

Schema validation is executed **inside the engine**.


In [8]:
solvency_results = solvency_engine(ratios_flat)


✅ solvency_engine output validated successfully.


## Review Solvency Engine Output

We generate a **human-readable table** to verify:

- Severity labeling  
- Explanation text  
- One record per company-year


In [9]:
pd.DataFrame([
    {
        "Company": r["Company"],
        "Year": r["Year"],
        "Severity": r["severity"],
        "Explanation": r["explanation"]
    }
    for r in solvency_results
])


,Company,Year,Severity,Explanation
0,Acme Manufacturing Ltd,2020,watch,Capital structure shows solvency risk.
1,Acme Manufacturing Ltd,2021,watch,Capital structure shows solvency risk.
2,Acme Manufacturing Ltd,2022,stable,Solvency position acceptable.
3,Acme Manufacturing Ltd,2023,stable,Solvency position acceptable.
4,Banyan Retail Co,2020,stable,Solvency position acceptable.
5,Banyan Retail Co,2021,watch,Capital structure shows solvency risk.
6,Banyan Retail Co,2022,watch,Capital structure shows solvency risk.
7,Banyan Retail Co,2023,watch,Capital structure shows solvency risk.
8,Coastal Tech Ltd,2020,stable,Solvency position acceptable.
9,Coastal Tech Ltd,2021,stable,Solvency position acceptable.


## Output Contract Validation

Each Solvency Engine record conforms to the AFAP engine schema:

- One record per company-year  
- `metrics` dictionary with numeric ratios  
- Boolean flags for risk indicators  
- Deterministic severity label  
- Plain-language explanation

This structure allows:

- Seamless ingestion by the **AFAP orchestrator**  
- Safe downstream **LLM interpretation**


## Validation Conclusion

✔ Solvency metrics are correctly sourced from the Ratio Engine  
✔ Risk flags trigger deterministically  
✔ Severity labels behave as designed  
✔ Schema validation passes inside the engine  
✔ Engine output is orchestration-ready and AI-safe  

The AFAP Solvency Engine is **locked, auditable, and production-ready**.
